In [86]:
import functions as fn
import pandas as pd
from importlib import reload
from tqdm import tqdm
from termcolor import colored
import matplotlib.pyplot as plt
from ooipy.tools import ooiplotlib as ooiplt
import ooipy
import numpy as np
from datetime import datetime,timedelta

In [87]:
path_to_write='E:/Acer backup 3/internships and jobs/UW/Reader grader and TA/Shima Abadi/Data/Benchmark_data/No_ships/'

In [88]:
def get_no_ships_slope_hydrate (df,rad,min_d,freq): #optimised version
    
    
    df=df.sort_values(by=['TIMESTAMP UTC'],ascending=True)
    df_time_grp=df.groupby(pd.Grouper(key="TIMESTAMP UTC", freq=freq)).agg({'TIMESTAMP UTC':['min','max'],
                                                                               'MMSI':'count',
                                                                              'distance(in km) oregon':'min',
                                                                              'distance(in km) hydrate':'min'}).reset_index()
    df_time_grp.columns=['TIMESTAMP UTC','min_time','max_time','count','min_slope_dist','min_hydrate_dist']
    
    df_time_grp['start_time']=df_time_grp['TIMESTAMP UTC']
    df_time_grp['end_time']=df_time_grp['TIMESTAMP UTC']+timedelta(minutes=min_d)
    df_time_grp['slope_noship']=[1 if x>rad else 0 for x in df_time_grp['min_slope_dist'] ]
    df_time_grp['hydrate_noship']=[1 if x>rad else 0 for x in df_time_grp['min_hydrate_dist'] ]
    df_time_grp['YY-MM']=df_time_grp['TIMESTAMP UTC'].dt.strftime('%Y-%m')
#     df_time_grp1=df_time_grp[(df_time_grp['slope_noship']==1) | (df_time_grp['hydrate_noship']==1)]

    return df_time_grp

In [89]:
def get_no_ships_axial(df,rad,min_d,freq): #optimised version
    
    
    df=df.sort_values(by=['TIMESTAMP UTC'],ascending=True)
    df_time_grp=df.groupby(pd.Grouper(key="TIMESTAMP UTC", freq=freq)).agg({'TIMESTAMP UTC':['min','max'],
                                                                               'MMSI':'count','distance(in km) axial':'min',
                                                                              'distance(in km) central cald':'min',
                                                                              'distance(in km) eastern cald':'min'}).reset_index()
    df_time_grp.columns=['TIMESTAMP UTC','min_time','max_time','count','min_axial_dist','min_central_dist','min_eastern_dist']
    
    df_time_grp['start_time']=df_time_grp['TIMESTAMP UTC']
    df_time_grp['end_time']=df_time_grp['TIMESTAMP UTC']+timedelta(minutes=min_d)
    df_time_grp['axial_noship']=[1 if x>rad else 0 for x in df_time_grp['min_axial_dist'] ]
    df_time_grp['central_noship']=[1 if x>rad else 0 for x in df_time_grp['min_central_dist'] ]
    df_time_grp['eastern_noship']=[1 if x>rad else 0 for x in df_time_grp['min_central_dist'] ]
    df_time_grp['YY-MM']=df_time_grp['TIMESTAMP UTC'].dt.strftime('%Y-%m')
#     df_time_grp1=df_time_grp[(df_time_grp['axial_noship']==1) | (df_time_grp['central_noship']==1) | (df_time_grp['eastern_noship']==1) ]

    return df_time_grp

In [90]:
def get_spectrogram_axial(df,hydro_idx,fmin=10,fmax=90,rad=10):
    print('Saving spectrograms')
    if hydro_idx==1:
        hydrophone_idx=1
        hydro='Axial_Base'
        df2=df[df.axial_noship==1]
    elif hydro_idx==2:
        hydrophone_idx=2
        hydro='Central_Caldera'
        df2=df[df.central_noship==1]
    elif hydro_idx==3:
        hydrophone_idx=3
        hydro='Eastern_Caldera'
        df2=df[df.eastern_noship==1]
        
    df_time_grp=df2.groupby('YY-MM', group_keys=False).apply(lambda x: x.sample(count_per_month))  
    
    print(df_time_grp.head())
    
    for i in tqdm(range(len(df_time_grp))):
        min_time=df_time_grp.start_time.iloc[i]
        max_time=df_time_grp.end_time.iloc[i]
        
        spectrogram=fn.get_spectrogram_data(hydrophone_idx,min_time,max_time,fmin=fmin,fmax=fmax)
        if (spectrogram==None):
            # print('data trace is none. Continuing to next')
            pass
        else:
            if (spectrogram.values.shape[0]<2) | (spectrogram.values.shape[1]<2):
                pass
            else:
                #spectrogram.visualize(save_spec=True,plot_spec=False,filename=path_to_write+hydro+'/Spectrogram'+'/isolated_ships_'+str(inner_rad)+'_'+ str(outer_rad)+'_'+str(i)+'.png')
                ooiplt.plot_spectrogram(spectrogram,plot=False,save=True,fmin=fmin,fmax=fmax,filename=path_to_write+hydro+'/Spectrogram'+'/isolated_ships_'+str(rad)+'_'+str(i)+'.png')

In [91]:
def get_spectrogram_slope_hydrate(df,hydro_idx,fmin=10,fmax=90,rad=10):
    print('Saving spectrograms')
    
    if hydro_idx==4:
        hydrophone_idx=4
        hydro='Oregon_Slope'
        df2=df[df.slope_noship==1]
    elif hydro_idx==5:
        hydrophone_idx=5
        hydro='hydrate_noship'
        df2=df[df.hydrate_noship==1]
          
    df_time_grp=df2.groupby('YY-MM', group_keys=False).apply(lambda x: x.sample(count_per_month))          
    for i in tqdm(range(len(df_time_grp))):
        min_time=df_time_grp.start_time.iloc[i]
        max_time=df_time_grp.end_time.iloc[i]
        spectrogram=fn.get_spectrogram_data(hydrophone_idx,min_time,max_time,fmin=fmin,fmax=fmax)
        if (spectrogram==None):
            # print('data trace is none. Continuing to next')
            pass
        else:
            if (spectrogram.values.shape[0]<2) | (spectrogram.values.shape[1]<2):
                pass
            else:
                #spectrogram.visualize(save_spec=True,plot_spec=False,filename=path_to_write+hydro+'/Spectrogram'+'/isolated_ships_'+str(inner_rad)+'_'+ str(outer_rad)+'_'+str(i)+'.png')
                ooiplt.plot_spectrogram(spectrogram,plot=False,save=True,fmin=fmin,fmax=fmax,filename=path_to_write+hydro+'/Spectrogram'+'/isolated_ships_'+str(rad)+'_'+str(i)+'.png')

### Getting no ships data for axial, central and eastern caldera

In [96]:
df = pd.read_csv('E:/Acer backup 3/internships and jobs/UW/Reader grader and TA/Shima Abadi/Data/ais_2014B_2020B_V2.csv',sep = ',')

df['TIMESTAMP UTC']=df['TIMESTAMP UTC'].astype('datetime64[ns]')

count_iso_instances=11387
count_noship_instances=int(count_iso_instances+count_iso_instances*0.40)
min_date=df['TIMESTAMP UTC'].min()
max_date=df['TIMESTAMP UTC'].max()
num_months=int((max_date-min_date).total_seconds()/2629800)
count_per_month=int(count_noship_instances/num_months)

In [97]:
#removing airgun experiment period
start_time=datetime(2019,7,1,0,0,0)
end_time=datetime(2019,8,1,0,0,0)


In [98]:
df=df[(df['TIMESTAMP UTC']<start_time) | (df['TIMESTAMP UTC'] > end_time)]
df.head()

,Unnamed: 0.1,Unnamed: 0,MMSI,SHIPNAME,VESSEL TYPE,STATUS,SPEED (KNOTSx10),LAT,LON,COURSE,...,central_caldera_Loc,eastern_caldera_Lat,eastern_caldera_Lon,eastern_caldera_Loc,distance(in km) axial,distance(in km) central cald,distance(in km) eastern cald,bearing_axial,bearing_eastern_cald,bearing_central_cald
0,0,0,209605000,AKILI,Bulk Carrier,0,108,46.09859,-129.6550,83,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",32.25,31.72,30.37,193.760689,234.515164,239.809636
1,1,1,256832000,NaN,NaN,0,115,45.07486,-128.9430,178,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",104.04,128.46,125.45,322.804182,320.490292,320.057435
2,2,2,352358000,ANNA G,Bulk Carrier,0,132,45.86138,-130.5627,86,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",63.02,44.21,46.50,94.234035,78.935325,76.159593
3,3,3,356566000,GLOBAL SAIKAI,General Cargo,0,104,46.50261,-129.0129,272,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",95.33,98.03,97.02,217.082385,230.114815,231.878577
4,4,4,477293500,JIN XIU FENG,Bulk Carrier,0,112,46.74633,-129.5912,299,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",104.08,93.66,94.37,186.960616,198.291220,200.167826


In [99]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'MMSI', 'SHIPNAME', 'VESSEL TYPE',
       'STATUS', 'SPEED (KNOTSx10)', 'LAT', 'LON', 'COURSE', 'HEADING',
       'TIMESTAMP UTC', 'LENGTH', 'Year', 'axial_Lat', 'axial_Lon',
       'axial_Loc', 'ship_Loc', 'central_caldera_Lat', 'central_caldera_Lon',
       'central_caldera_Loc', 'eastern_caldera_Lat', 'eastern_caldera_Lon',
       'eastern_caldera_Loc', 'distance(in km) axial',
       'distance(in km) central cald', 'distance(in km) eastern cald',
       'bearing_axial', 'bearing_eastern_cald', 'bearing_central_cald'],
      dtype='object')

In [100]:
#Usig
no_ships_axial=get_no_ships_axial(df,30,10,'10min')

In [101]:
no_ships_axial.columns

Index(['TIMESTAMP UTC', 'min_time', 'max_time', 'count', 'min_axial_dist',
       'min_central_dist', 'min_eastern_dist', 'start_time', 'end_time',
       'axial_noship', 'central_noship', 'eastern_noship', 'YY-MM'],
      dtype='object')

In [102]:
no_ships_axial.tail()

,TIMESTAMP UTC,min_time,max_time,count,min_axial_dist,min_central_dist,min_eastern_dist,start_time,end_time,axial_noship,central_noship,eastern_noship,YY-MM
315641,2020-12-31 23:10:00,2020-12-31 23:10:14,2020-12-31 23:19:05,11,60.66,81.18,78.62,2020-12-31 23:10:00,2020-12-31 23:20:00,1,1,1,2020-12
315642,2020-12-31 23:20:00,2020-12-31 23:20:27,2020-12-31 23:28:27,10,57.06,78.34,75.69,2020-12-31 23:20:00,2020-12-31 23:30:00,1,1,1,2020-12
315643,2020-12-31 23:30:00,2020-12-31 23:31:52,2020-12-31 23:39:34,8,53.45,75.56,72.81,2020-12-31 23:30:00,2020-12-31 23:40:00,1,1,1,2020-12
315644,2020-12-31 23:40:00,2020-12-31 23:40:54,2020-12-31 23:49:26,9,50.69,73.44,70.61,2020-12-31 23:40:00,2020-12-31 23:50:00,1,1,1,2020-12
315645,2020-12-31 23:50:00,2020-12-31 23:50:43,2020-12-31 23:58:58,11,47.86,71.30,68.39,2020-12-31 23:50:00,2021-01-01 00:00:00,1,1,1,2020-12


### Get spectrogram images

### Axial Base

In [ ]:
get_spectrogram_axial(no_ships_axial,1,rad=6)

Saving spectrograms
           TIMESTAMP UTC            min_time            max_time  count  \
3315 2015-01-24 00:50:00 2015-01-24 00:57:19 2015-01-24 00:57:19      1   
1704 2015-01-12 20:20:00 2015-01-12 20:23:00 2015-01-12 20:23:00      2   
3325 2015-01-24 02:30:00 2015-01-24 02:36:03 2015-01-24 02:36:03      1   
3180 2015-01-23 02:20:00 2015-01-23 02:26:56 2015-01-23 02:29:17      4   
1288 2015-01-09 23:00:00 2015-01-09 23:05:00 2015-01-09 23:05:00      2   

      min_axial_dist  min_central_dist  min_eastern_dist          start_time  \
3315           69.57             75.49             74.00 2015-01-24 00:50:00   
1704           49.42             73.88             70.84 2015-01-12 20:20:00   
3325           94.57            104.14            102.94 2015-01-24 02:30:00   
3180           41.98             32.48             32.46 2015-01-23 02:20:00   
1288           41.60             36.17             35.62 2015-01-09 23:00:00   

                end_time  axial_noship  central_

  0%|                                                                                        | 0/15691 [00:00<?, ?it/s]

OO.AXBA1.AXBA1.HDH | 2015-01-24T00:50:00.000000Z - 2015-01-24T01:00:00.000000Z | 200.0 Hz, 120001 samples


  0%|                                                                             | 1/15691 [00:03<14:55:06,  3.42s/it]

  0%|                                                                             | 2/15691 [00:05<11:05:30,  2.55s/it]

...

 28%|████████████████████▎                                                   | 4420/15691 [2:54:26<11:11:45,  3.58s/it]

OO.AXBA1.AXBA1.HDH | 2016-09-13T00:30:00.000000Z - 2016-09-13T00:40:00.000000Z | 200.0 Hz, 120001 samples


 28%|████████████████████▎                                                   | 4421/15691 [2:54:32<12:56:51,  4.14s/it]

OO.AXBA1.AXBA1.HDH | 2016-09-14T10:20:00.000000Z - 2016-09-14T10:30:00.000000Z | 200.0 Hz, 120001 samples


 28%|████████████████████▎                                                   | 4422/15691 [2:54:41<18:00:58,  5.76s/it]

### Central Caldera

In [ ]:
get_spectrogram_axial(no_ships_axial,2,rad=6)

### Eastern Caldera

In [ ]:
get_spectrogram_axial(no_ships_axial,3,rad=6)

### Getting no ships data for oregon slope and southern hydrate

In [ ]:
df = pd.read_csv('E:/Acer backup 3/internships and jobs/UW/Reader grader and TA/Shima Abadi/Data/oregon_hydrate.csv',sep = ',')

df['TIMESTAMP UTC']=df['TIMESTAMP UTC'].astype('datetime64[ns]')

count_iso_instances=6000
count_noship_instances=int(count_iso_instances+count_iso_instances*0.40)
min_date=df['TIMESTAMP UTC'].min()
max_date=df['TIMESTAMP UTC'].max()
num_months=int((max_date-min_date).total_seconds()/2629800)
count_per_month=int(count_noship_instances/num_months)

In [ ]:
no_ships_slope_hydrate=get_no_ships_slope_hydrate(df,30,10,'10min')

In [ ]:
no_ships_slope_hydrate

In [ ]:
df.columns

### Get spectrogram images

### Oregon Slope

In [ ]:
get_spectrogram_slope_hydrate(no_ships_slope_hydrate,4,rad=6)

### Southern hydrate

In [ ]:
get_spectrogram_slope_hydrate(no_ships_slope_hydrate,5,rad=6)